In [21]:

import numpy as np 
import pandas as pd 




In [22]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import (StratifiedKFold, train_test_split, 
                                     RandomizedSearchCV, GridSearchCV)
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score




In [23]:
train = pd.read_csv('../input/amazon-employee-access-challenge/train.csv')
test = pd.read_csv('../input/amazon-employee-access-challenge/test.csv')

In [24]:
train

In [25]:
test

In [26]:
train['RESOURCE'].value_counts()

In [27]:
train['ACTION']=train['ACTION'].astype('float64')

In [28]:
X=train.drop(columns='ACTION').to_numpy()
y=train['ACTION'].to_numpy()

In [29]:
train = train.astype({"ACTION":'int64'})

In [30]:
train.dtypes

In [31]:
train_corr=train.corr()
train_corr['ACTION'].sort_values(ascending=False)

In [32]:
train=train.drop(columns='ROLE_TITLE')
test=test.drop(columns='ROLE_TITLE')

In [33]:
sns.countplot(train['ACTION'])


In [34]:
sns.pairplot(data=train, kind='scatter')


In [35]:
sns.relplot(data=train, x="ROLE_FAMILY", y="RESOURCE", hue="ACTION")


In [36]:
#  sns.swarmplot(x='ACTION', y='ROLE_CODE', data=train)


In [37]:
corr_matrix=train.corr()
corr_matrix['ACTION']

In [38]:
train['ACTION'].value_counts()

In [39]:
l_enc = LabelEncoder()
X=train.drop(columns='ACTION').to_numpy()
y=train['ACTION'].to_numpy()
y = l_enc.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, stratify=y)

trained= pd.DataFrame(X_train)
trained.columns= train.columns[1:]



In [40]:
# xgb=XGBClassifier(subsample=0.1)


# param_grid={
#     "xgb__n_estimators":np.arange(100,1500,100),
#     "xgb__max_depth": np.arange(3,11,2),
#     "xgb__learning_rate": np.arange(0.001,0.1)
# }

# random_search= RandomizedSearchCV(
#                          estimator=xgb,
#                          param_distributions=param_grid,
#                          scoring='roc_auc',
#                          verbose=10,
#                          n_iter=100,
#                          n_jobs=4,
#                          )

# random_search.fit(X_train,y_train)

# print(random_search.best_score_, random_search.best_estimator_.get_params())

In [41]:
test=test.drop(columns='id')
X=test.to_numpy()

In [42]:
xgb=XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type='gain', interaction_constraints='',
              learning_rate=0.3, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1)

xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
fold_auc = roc_auc_score(y_test, y_pred)
fold_auc

In [43]:
predictions= xgb.predict_proba(X)
predictions

In [44]:
sub=pd.read_csv("../input/amazon-employee-access-challenge/sampleSubmission.csv")
sub.Id=np.arange(1,len(test)+1)
sub.Action=predictions[:,1]

sub

In [45]:
sub.to_csv('blablabla.csv', index=False)
sub.head()